In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from model.stft import AudioPreprocessor
import sounddevice as sd # 마이크 입력을 위한 라이브러리

In [17]:
print(sd.query_devices())

   0 Microsoft 사운드 매퍼 - Input, MME (2 in, 0 out)
>  1 마이크(Realtek USB MIC), MME (1 in, 0 out)
   2 마이크(Steam Streaming Microphone), MME (8 in, 0 out)
   3 Microsoft 사운드 매퍼 - Output, MME (0 in, 2 out)
<  4 스피커(High Definition Audio Devic, MME (0 in, 6 out)
   5 스피커(Steam Streaming Speakers), MME (0 in, 8 out)
   6 스피커(Steam Streaming Microphone), MME (0 in, 8 out)
   7 Display(2- High Definition Audi, MME (0 in, 2 out)
   8 디지털 출력(High Definition Audio De, MME (0 in, 2 out)
   9 주 사운드 캡처 드라이버, Windows DirectSound (2 in, 0 out)
  10 마이크(Realtek USB MIC), Windows DirectSound (1 in, 0 out)
  11 마이크(Steam Streaming Microphone), Windows DirectSound (8 in, 0 out)
  12 주 사운드 드라이버, Windows DirectSound (0 in, 2 out)
  13 스피커(High Definition Audio Device), Windows DirectSound (0 in, 6 out)
  14 스피커(Steam Streaming Speakers), Windows DirectSound (0 in, 8 out)
  15 스피커(Steam Streaming Microphone), Windows DirectSound (0 in, 8 out)
  16 Display(2- High Definition Audio Device), Windows DirectSound (

In [18]:
sd.default.device = (1, None)

In [19]:
MODEL_PATH = "./result/best.pt"
CLASS_NAME = ["danger", "fire", "gas", "non", "tsunami"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 하이퍼 파라미터
CLASS_NUM = len(CLASS_NAME)
SAMPLE_RATE = 16000
DURATION = 1
NUM_SAMPLES = SAMPLE_RATE * DURATION
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 1e-4
WINDOW_SIZE = 512
HOP_SIZE = 160
MEL_BINS = 64
FMIN = 50
FMAX = 8000

In [ ]:
def preprocess(audio_np, audio_preprocessor):
    # 2채널 이상일때 1채널로 변환
    if audio_np.ndim > 1:
        audio_np = audio_np.mean(axis=-1)

    # numpy 배열을 torch 텐서로 변환, 배치 차원 추가: (1, num_samples)
    waveform = torch.tensor(audio_np, dtype=torch.float32).unsqueeze(0)

    # 길이가 부족할 경우 뒤에 0을 채움
    if waveform.shape[1] < NUM_SAMPLES:
        waveform = F.pad(waveform, (0, NUM_SAMPLES - waveform.shape[1]))
    # 길이가 길 경우 자름
    else:
        waveform = waveform[:, :NUM_SAMPLES]

    # 절대값 정규화
    waveform = waveform / (waveform.abs().max() + 1e-9)

    # input: wavefrom (1, length)
    with torch.no_grad():
        logmel = audio_preprocessor(waveform) # (1, 1, time, mel_bins)
        # print(f"logmel: {logmel.shape}")

    return logmel

def predict(model, logmel_tensor, device=DEVICE, class_name=CLASS_NAME):
    logmel_tensor = logmel_tensor.to(device)

    with torch.no_grad():
        output_dict = model(logmel_tensor)
        prob = output_dict["clipwise_output"] # 클래스별 확률 출력
        prob_np = prob.cpu().numpy()[0] # numpy 배열로 변환 (shape: [클래스 수])

        # 0.5를 기준으로 예측된 클래스 선택
        pred_tensor = (prob_np > 0.5).astype(int)
        pred_labels = [class_name[i] for i, p in enumerate(pred_tensor) if p == 1]

        # 어떤 클래스도 0.5 이상이 아니면 가장 확률 높은 클래스 하나 선택
        if not pred_labels:
            highest_prob_idx = int(np.argmax(prob_np))
            return [class_name[highest_prob_idx]], prob_np
        
        return pred_labels, prob_np # 예측된 라벨 리스트, 클래스별 확률 반환

In [21]:
model = torch.load(MODEL_PATH, map_location=torch.device(DEVICE), weights_only=False)
model.to(DEVICE)
model.eval()

audio_preprocessor = AudioPreprocessor(
    sample_rate=SAMPLE_RATE,
    window_size=WINDOW_SIZE,
    hop_size=HOP_SIZE,
    mel_bins=MEL_BINS,
    fmin=FMIN,
    fmax=FMAX
)

In [22]:
print(model)

Cnn14(
  (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block3): ConvBlock(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1),

In [23]:
print("실시간 마이크 입력을 시작합니다.")

try:
    # 실시간 마이크 입력 루프
    while True:
        # 오디오를 실시간으로 마이크에서 녹음
        audio_np = sd.rec(frames=NUM_SAMPLES, # 녹음할 샘플 수 (1초 분량)
                          samplerate=SAMPLE_RATE, # 샘플링 레이트
                          channels=1, # 단일 채널 (모노)
                          dtype="float32" # float32 타입으로 녹음
                          )
        sd.wait() # 녹음 완료까지 대기

        audio_np = np.squeeze(audio_np, axis=-1) # 녹음된 오디오: (16000, 1) → (16000,)으로 reshape
        audio_tensor = preprocess(audio_np, audio_preprocessor) # 전처리 → (1, 16000) 텐서로 변환
        pred_labels, class_prob = predict(model, audio_tensor) # 예측

        print(f"예측 결과: {', '.join(pred_labels)}")
        for i, name in enumerate(CLASS_NAME):
            print(f" - {name}: {class_prob[i]:.4f}")

except KeyboardInterrupt as e:
    print("\n실시간 마이크 입력을 종료합니다.\n", e)

실시간 마이크 입력을 시작합니다.
logmel: torch.Size([1, 1, 101, 64])
예측 결과: non
 - danger: 0.0021
 - fire: 0.0006
 - gas: 0.0009
 - non: 0.9732
 - tsunami: 0.0086
logmel: torch.Size([1, 1, 101, 64])
예측 결과: non
 - danger: 0.0035
 - fire: 0.0011
 - gas: 0.0094
 - non: 0.9786
 - tsunami: 0.0198
logmel: torch.Size([1, 1, 101, 64])
예측 결과: non
 - danger: 0.0062
 - fire: 0.0008
 - gas: 0.0037
 - non: 0.9840
 - tsunami: 0.0248
logmel: torch.Size([1, 1, 101, 64])
예측 결과: non
 - danger: 0.0006
 - fire: 0.0015
 - gas: 0.0035
 - non: 0.9796
 - tsunami: 0.0118
logmel: torch.Size([1, 1, 101, 64])
예측 결과: non
 - danger: 0.0114
 - fire: 0.0097
 - gas: 0.0557
 - non: 0.8198
 - tsunami: 0.0283
logmel: torch.Size([1, 1, 101, 64])
예측 결과: non
 - danger: 0.0126
 - fire: 0.0074
 - gas: 0.0051
 - non: 0.8047
 - tsunami: 0.0771
logmel: torch.Size([1, 1, 101, 64])
예측 결과: non
 - danger: 0.0020
 - fire: 0.0015
 - gas: 0.0303
 - non: 0.9015
 - tsunami: 0.0117
logmel: torch.Size([1, 1, 101, 64])
예측 결과: non
 - danger: 0.0043
 - fir